In [1]:
%load_ext autoreload
%autoreload 2

Download if file is not in folder.

In [2]:
from litedict import SQLDict

In [3]:
TEST_1 = "key_test_1"
TEST_2 = "key_test_2"

Basic functionality

In [4]:
d = SQLDict(":memory:")

d[TEST_1] = "asdfoobar"

assert d[TEST_1] == "asdfoobar"

del d[TEST_1]

assert d.get(TEST_1, None) is None

Glob matching

In [5]:
d[TEST_1] = "asdfoobar"

d[TEST_2] = "foobarasd"

d["key_testx_3"] = "barasdfoo"

assert d.glob("key_test*") == ["asdfoobar", "foobarasd", "barasdfoo"]

assert d.glob("key_test_?") == ["asdfoobar", "foobarasd"]

assert d.glob("key_tes[tx]*") == ["asdfoobar", "foobarasd", "barasdfoo"]

Numbers

In [6]:
d[TEST_1] = 1

d[TEST_2] = 2

assert d[TEST_1] + d[TEST_2] == 3

## Benchmarks

In [7]:
from string import ascii_lowercase, printable
from random import choice
import random


def random_string(string_length=10, fuzz=False, space=False):
    """Generate a random string of fixed length """
    letters = ascii_lowercase
    letters = letters + " " if space else letters
    if fuzz:
        letters = printable
    return "".join(choice(letters) for i in range(string_length))

In [8]:
import gc

import pickle

import json

**Pickle**

In [9]:
d = SQLDict(
    ":memory:",
    encoder=lambda x: pickle.dumps(x).hex(),
    decoder=lambda x: pickle.loads(bytes.fromhex(x)),
)

In [10]:
gc.collect()

198

In [11]:
%%timeit -n20000 -r10

d[random_string(8)] = random_string(50)

d.get(random_string(8), None)

82 µs ± 6.89 µs per loop (mean ± std. dev. of 10 runs, 20000 loops each)


**Noop**

In [12]:
d = SQLDict(
    ":memory:",
    encoder=lambda x: x,
    decoder=lambda x: x,
)

In [13]:
gc.collect()

69

In [14]:
%%timeit -n20000 -r10

d[random_string(8)] = random_string(50)

d.get(random_string(8), None)

79.8 µs ± 18.2 µs per loop (mean ± std. dev. of 10 runs, 20000 loops each)


**JSON**

In [15]:
d = SQLDict(
    ":memory:",
    encoder=lambda x: json.dumps(x),
    decoder=lambda x: json.loads(x),
)

In [16]:
gc.collect()

69

In [17]:
%%timeit -n20000 -r10

d[random_string(8)] = random_string(50)

d.get(random_string(8), None)

85.4 µs ± 8.01 µs per loop (mean ± std. dev. of 10 runs, 20000 loops each)


**Pickle Python obj**

In [18]:
d = SQLDict(
    ":memory:",
    encoder=lambda x: pickle.dumps(x).hex(),
    decoder=lambda x: pickle.loads(bytes.fromhex(x)),
)

In [19]:
gc.collect()

69

In [20]:
class C:
    def __init__(self, x):
        self.x = x

    def pp(self):
        return x

    def f(self):
        def _f(y):
            return y * self.x ** 2

        return _f

In [21]:
%%timeit -n20000 -r10

d[random_string(8)] = C(random.randint(1, 200))

d.get(random_string(8), None)

40.5 µs ± 6.47 µs per loop (mean ± std. dev. of 10 runs, 20000 loops each)


**Dictionary**

In [22]:
d = {}

In [23]:
gc.collect()

91

In [24]:
%%timeit -n20000 -r10

d[random_string(8)] = random_string(50)

d.get(random_string(8), None)

55.3 µs ± 4.13 µs per loop (mean ± std. dev. of 10 runs, 20000 loops each)
